<a href="https://colab.research.google.com/github/shivamsonawane/Sentiment-Clustering/blob/main/LDA_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:





 # !pip install stop_words
# !pip install language_detector
# !pip install symspellpy
# !pip install umap
# !pip install sentence_transformers
# !pip install 'umap-learn==0.3.10'

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import pandas as pd
import re
import nltk
nltk.download('popular')
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# cohernece
from collections import Counter
from sklearn.metrics import silhouette_score
from gensim.models.coherencemodel import CoherenceModel    
import umap
from wordcloud import WordCloud
import os
import matplotlib.pyplot as plt

# lda
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
from datetime import datetime
import pickle

# autoencoder
import tensorflow
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

### Preprocessing

In [ ]:


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()


# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English', 'French', 'Spanish', 'Chinese'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
p_stemmer = PorterStemmer()


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) for word in w_list]


# filtering out stop words
# create English stop words list

stop_words = (list(
    set(get_stop_words('en'))
    | set(get_stop_words('es'))
    | set(get_stop_words('de'))
    | set(get_stop_words('it'))
    | set(get_stop_words('ca'))
    # |set(get_stop_words('cy'))
    | set(get_stop_words('pt'))
    # |set(get_stop_words('tl'))
    | set(get_stop_words('pl'))
    # |set(get_stop_words('et'))
    | set(get_stop_words('da'))
    | set(get_stop_words('ru'))
    # |set(get_stop_words('so'))
    | set(get_stop_words('sv'))
    | set(get_stop_words('sk'))
    # |set(get_stop_words('cs'))
    | set(get_stop_words('nl'))
    # |set(get_stop_words('sl'))
    # |set(get_stop_words('no'))
    # |set(get_stop_words('zh-cn'))
))


def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
    w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list


def preprocess(docs):
    """
    objective: preprocess the raw text and return clean text and tokenized text
    :param docs: input raw text
    :return: sentences and clean token
    """
    # calculate total len for document
    n_docs = len(docs)
    print('[INFO] Preprocessing {} raw texts ...'.format(n_docs))
    sentences = []  # sentence level preprocessed
    token_lists = []  # word level preprocessed

    for idx in range(n_docs):
        sentence = preprocess_sent(docs[idx])
        token_list = preprocess_word(sentence)
        if token_list:
            sentences.append(sentence)
            token_lists.append(token_list)
        print('{} %'.format(str(np.round((idx + 1) / n_docs * 100, 2))), end='\r')
    print("[INFO] Total preprocessed sentences: {}".format(len(sentences)))
    print("[INFO] Total Tokenized lists: {}".format(len(token_lists)))
    print('[INFO] Preprocessing raw texts. Done!')
    return sentences, token_lists


## Utility

In [ ]:

def get_topic_words(token_lists, labels, k=None):
    """
    objective : get top words within each topic from clustering results
    :param token_lists: all tokens for each documents
    :param labels: labels associated with that documents
    :param k:
    :return:
    """
    print("[INFO] Getting top words for each topic from clustering result")
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))
    return topics


def get_coherence(model, token_lists, measure='c_v'):
    """
    objective : Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        print("[INFO] Calculating Coherence Score For LDA...")
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
        print("[INFO] Coherence Score: {}".format(cm))
    else:
        print("[INFO] Calculating Coherence Score for others (lda_bert, bert, tfidf)")
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
        cs = cm.get_coherence()
        print("[INFO] Coherence Score: {}".format(cs))
    return cs


def get_silhouette(model):
    """
    objective: Get silhouette score from model
    :param model: Topic_Model object
    :return: silhouette score
    """
    if model.method == 'LDA':
        print("[INFO] silhouette scores not applicable for LDA")
        return
    print("[INFO] Calculating silhouette scores for clusters..")
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    sc = silhouette_score(vec, lbs)
    print("[INFO] Silhouette scores: {}".format(sc))
    return sc


## VISULISATION

In [ ]:

def plot_proj(embedding, lbs):
    """
    Objective: Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend()


def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: Topic_Model object
    """
    if model.method == 'LDA':
        return
    reducer = umap.UMAP()
    print('[INFO] Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('[INFO] Calculating UMAP projection. Done!')
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = 'images/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig(dr + '/2D_vis')

def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: Topic_Model object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return
    print('[INFO] Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = 'images/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig(dr + '/Topic' + str(topic) + '_wordcloud')
    print('[INFO] Getting wordcloud for topic {}. Done!'.format(topic))


### Autoencoder

In [ ]:


class Autoencoder:
    """
    Autoencoder for learning latent space representation
    architecture simplified for only one hidden layer
    """

    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.his = None

    def _compile(self, input_dim):
        """
        compile the computational graph
        """
        input_vec = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder = Model(input_vec, decoded)
        self.encoder = Model(input_vec, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoder_layer = self.autoencoder.layers[-1]
        self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
        self.autoencoder.compile(optimizer='adam', loss=tensorflow.keras.losses.mean_squared_error)

    def fit(self, X):
        if not self.autoencoder:
            self._compile(X.shape[1])
        X_train, X_test = train_test_split(X)
        self.his = self.autoencoder.fit(X_train, X_train,
                                        epochs=self.epochs,
                                        batch_size=self.batch_size,
                                        shuffle=True,
                                        validation_data=(X_test, X_test), verbose=0)






### BASE TM

In [ ]:

# define model object
class Topic_Model:
    def __init__(self, k=10, method='TFIDF'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
            raise Exception('Invalid method!')

        self.k = k
        self.dictionary = None
        self.corpus = None
        # self.stopwords = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for relative importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        print("[INFO] Topic Model object created for method: {}".format(self.method))

    def vectorize(self, sentences, token_lists, method=None):
        """
        objective:  Get vector representations from selected methods
        :param sentences: input sentences required for clustering
        :param token_lists: tokens list required for Topic Modeling
        :param method: method for Topic Modeling
        :return: None
        """

        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('[INFO] Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('[INFO] Getting vector representations for TF-IDF. Done!')
            print("[INFO] TFIDF vector shape: {}".format(vec.shape))
            return vec

        elif method == 'LDA':
            print('[INFO] Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)

            def get_vec_lda(model, corpus, k):
                """
                objective: Get the LDA vector representation (probabilistic topic assignments for all documents)
                :param model:
                :param corpus:
                :param k:
                :return: vec_lda with dimension: (n_doc * n_topic)
                """

                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('[INFO] Getting vector representations for LDA. Done!')
            print("[INFO] LDA Vector shape: {}".format(vec.shape))
            return vec

        elif method == 'BERT':

            print('[INFO] Getting vector representations for BERT ...')
            from sentence_transformers import SentenceTransformer
            model = SentenceTransformer('bert-base-nli-max-tokens')
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('[INFO] Getting vector representations for BERT. Done!')
            print("[INFO] BERT vector shape: {}".format(vec.shape))
            return vec


        elif method == 'LDA_BERT':

            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            print("[INFO] Concatenating lda vector and bert vector...")
            vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
            print("[INFO] LDA_BERT vector shape : {}".format(vec_ldabert.shape))
            self.vec['LDA_BERT_FULL'] = vec_ldabert
            if not self.AE:
                self.AE = Autoencoder()
                print('[INFO] Fitting Autoencoder on LDA_BERT vec...')
                self.AE.fit(vec_ldabert)
                print('[INFO] Fitting Autoencoder Done!')
            vec = self.AE.encoder.predict(vec_ldabert)
            print("[INFO] Autoencoder vector shape: {}".format(vec.shape))
            return vec

    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        objective: Fit the topic model for selected method given the preprocessed data
        :param sentences: input sentences required for clustering
        :param token_lists: tokens list required for Topic Modeling
        :param method: method for Topic Modeling
        :param m_clustering: no of cluster
        :return: None
        """

        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('[INFO] Fitting LDA MODEL for Topic Modeling...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('[INFO] Fitting LDA Done!')
        else:
            print('[INFO] Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            print("[INFO] Fitting Kmeans clustering model..")
            self.cluster_model.fit(self.vec[method])
            print("[INFO] Fitting Kmeans clustering model Done!")
            print('[INFO] Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        objective:  Predict topics for new_documents
        :param sentences: input sentences required for clustering
        :param token_lists: tokens list required for Topic Modeling
        :param out_of_sample: True means prediction on new document
        :return:
        """

        # Default as False
        out_of_sample = out_of_sample is not None
        if out_of_sample:
            print("[INFO] Predicting on New documents..")
        else:
            print("[INFO] Predicting on same train data")

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs


## Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# set project directory
os.chdir("/content/drive/MyDrive/Sentiment_Project")

# read data
df1 = pd.read_excel("DATA/ukraine_tweets_sample_copy1.xlsx")
df2 = pd.read_excel("DATA/ukraine_tweets_sample_copy2.xlsx")
df3 = pd.read_excel("DATA/ukraine_tweets_sample_copy3.xlsx")

# concat all data files and remove irrelevent label
df = pd.concat([df1, df2, df3], axis=0)
df = df[df["Stance Label"]!="Irrelevant"].reset_index(drop=True)
df.to_csv("DATA/ALL_Combine_Data.csv", index=False)
print(df.shape)
df.head(10)

(4177, 4)


,id_str,created_at,full_text,Stance Label
0,1497026099494023173,2022-02-25 01:50:10,"Russian Rosgvardia troops, likely spetsnaz, cr...",Neutral
1,1497258644987002880,2022-02-25 17:14:13,@WrestlingWombat @HStefansonMB What is happeni...,ProUkraine
2,1497872993732628484,2022-02-27 09:55:25,Today on the campaign trail I had some tough c...,ProUkraine
3,1499163764951818241,2022-03-02 23:24:29,"ICYMI: n light of Russia sanctions, Fed chair ...",Neutral
4,1499467511087222789,2022-03-03 19:31:28,By now over a thousand refugees from #Ukraine ...,ProUkraine
5,1508139059427819522,2022-03-27 17:49:06,Whoever is coming up with reactive strategy by...,ProRussia
6,1508675556358774786,2022-03-29 05:20:57,He also writes about the possibility of Ukrain...,ProUkraine
7,1512504441324322834,2022-04-08 18:55:34,"At Horizon, we are deeply concerned and sadden...",ProUkraine
8,1515578470755192837,2022-04-17 06:30:40,#UkraineUnderAttack Join us in this Democracy ...,ProUkraine
9,1516846493637951489,2022-04-20 18:29:20,@RALee85 Russian tanks in cities where they’re...,ProUkraine


## Coherence Range

In [ ]:
# coherence range tune toppic model 
class CoherenceOverRange:
    def __init__(self, input_data_path='DATA/ALL_Combine_Data.csv', method='LDA_BERT',
                 ntopic_min=5, ntopic_max=26, step=1, save=True, output_data_path='DATA/output_lda_bert.csv'):
        self.input_data_path = input_data_path
        self.method = method
        self.ntopic_min = ntopic_min
        self.ntopic_max = ntopic_max
        self.step = step
        self.best_tm = None
        self.best_ntopic = None
        self.best_coherence = None
        self.save = save
        self.output_data_path = output_data_path

    def tune_model(self):
        documents = pd.read_csv(self.input_data_path)
        documents = documents.iloc[:, 0:2]
        samp_size = len(documents)
        documents.columns = ['index', 'abstract']
        documents.dropna(inplace=True)
        documents.abstract = documents.abstract.astype(str)
        rws = documents.abstract
        sentences, token_lists, idx_in = preprocess(rws, samp_size=samp_size)

        Coherence = []
        models = []
        ntopics = range(self.ntopic_min, self.ntopic_max, self.step)
        for ntopic in ntopics:
            # Define the topic model object
            tm = Topic_Model(k=ntopic, method=self.method)
            # Fit the topic model by chosen method
            tm.fit(sentences, token_lists)
            # Evaluate using metrics
            c = get_coherence(tm, token_lists, 'c_v')
            print('Number of Topics:', ntopic)
            print('Coherence:', c)
            Coherence.append(c)
            models.append(tm)

        # initialize data of lists.
        coherence_scores = {'n_topics': ntopics, 'coherence_' + self.method: Coherence}

        output = []
        scores = []
        for i in zip(models, Coherence, ntopics):
            output.append(i)
            scores.append(i[1])

        max_score = sorted(scores)[-1]
        for model, score, num_topics in output:
            if score == max_score:
                print(f'Selected LDA_BERT configuration: {num_topics} with Coherence score: {score}')
                self.best_tm = model
                self.best_ntopic = num_topics
                self.best_coherence = score

        # Store the best model version
        if self.save:
            pickle_file = open('../models/topic_model.pkl', 'wb')
            pickle.dump(self.best_tm, pickle_file)
            pickle_file.close()

        # Evaluate using metrics
        if self.method == "LDA":
            topics = []
            for topic in self.best_tm.ldamodel.print_topics(num_topics=self.best_ntopic):
                topic_list = [re.search(r"[a-zA-z]+", w).group(0) for w in topic[1].split('+')]
                topics.append(topic_list)
        else:
            topics = get_topic_words(token_lists, tm.cluster_model.labels_)
        labels = tm.predict(sentences, token_lists)

        predictions = documents.copy()
        predictions['topic_no'] = np.NaN
        # Mapping the queries with their labels
        count = 0
        for idx in idx_in:
            predictions['topic_no'][idx] = labels[count]
            count = count + 1

        mapped_predictions = predictions.groupby('topic_no')['abstract'].apply(list)
        mapped_predictions = mapped_predictions.reset_index()
        mapped_predictions.sort_values(by=['topic_no'], inplace=True)
        mapped_predictions['keywords_list'] = topics
        mapped_predictions.abstract = mapped_predictions.abstract.apply(lambda x: x[:500])
        if self.save:
            mapped_predictions.to_csv(self.output_data_path)
        return mapped_predictions, coherence_scores


# if __name__ == '__main__':
#     input_p = 'data/bbc-text.csv'
#     output_p = 'data/bbc-text-output.csv'
#     compute_co = CoherenceOverRange(input_data_path=input_p, output_data_path=output_p)
#     output_data, coherence_scores = compute_co.tune_model()


### Train Topic Modeling 

### main

In [ ]:

class LDABertModel:

    def __init__(self, input_data_path='"DATA/ALL_Combine_Data.csv"', method='LDA_BERT',
                 ntopic=10, save=True, output_data_path='DATA/output_bert.csv"'):
        self.input_data_path = input_data_path
        self.method = method
        self.ntopic = ntopic
        self.save = save
        self.output_data_path = output_data_path

    def train(self):
        # Reading the dataset
        documents = pd.read_csv(self.input_data_path)

        # keeping only the first two columns and 100 documents [index, text]
        documents = documents.iloc[:100, 0:2]

        # drop missing
        documents.dropna(inplace=True)
        print("[INFO] all columns: ", documents.columns)

        # rename columns
        documents.columns = ['index', 'text']

        # convert datatype to str
        documents.abstract = documents.text.astype(str)

        # select all raw text documents
        rws = documents.abstract

        # preprocess the text
        sentences, token_lists = preprocess(rws)

        # check for empty tokens
        empty = 0
        for t in token_lists:
            if len(t) == 0:
                empty += 1
        print("[INFO] Count For empty tokens: ", empty)

        # Define the topic model object
        tm = Topic_Model(k=self.ntopic, method=self.method)

        # Fit the topic model by chosen method
        tm.fit(sentences, token_lists)
        # Evaluate using metrics
        if self.method == "LDA":
            topics = []
            for topic in tm.ldamodel.print_topics(num_topics=self.ntopic):
                topic_list = [re.search(r"[a-zA-z]+", w).group(0) for w in topic[1].split('+')]
                topics.append(topic_list)
        else:
            topics = get_topic_words(token_lists, tm.cluster_model.labels_)
        labels = tm.predict(sentences, token_lists)

        c = get_coherence(tm, token_lists, 'c_v')
        s = get_silhouette(tm)
        print('[INFO] Number of Topics:', self.ntopic)

        # visualize and save img
        # visualize(tm)
        # for i in range(tm.k):
        #     get_wordcloud(tm, token_lists, i)

        predictions = documents.copy()
        predictions['topic_no'] = np.NaN
        # Mapping the queries with their labels

        predictions['topic_no'] = labels
        print("final output")
        print(predictions['topic_no'].value_counts())

        
        if self.save:
            predictions.to_csv(self.output_data_path)
        return predictions


# if __name__ == '__main__':
#     input_p = "../data/bbc-text.csv"
#     output_p = "../output/bbc-text_reviews.csv"
#     topic_model = LDABertModel(input_data_path=input_p, ntopic=5, output_data_path=output_p)
#     output_data = topic_model.train()


In [ ]:
input_data_path = "DATA/ALL_Combine_Data.csv"
method ='LDA_BERT'
ntopic = 5
save=True
output_data_path = "output/ALL_Combine_Data_Output.csv"

In [ ]:
# Reading the dataset
documents = pd.read_csv(input_data_path)

# keeping only the first two columns and 100 documents [index, text]
documents = documents[["id_str","full_text"]]

# drop missing
documents.dropna(inplace=True)
print("[INFO] all columns: ", documents.columns)

# rename columns
documents.columns = ['index', 'text']

# convert datatype to str
documents.abstract = documents.text.astype(str)

# select all raw text documents
rws = documents.abstract

# preprocess the text
sentences, token_lists = preprocess(rws)

# check for empty tokens
empty = 0
for t in token_lists:
    if len(t) == 0:
        empty += 1
print("[INFO] Count For empty tokens: ", empty)

[INFO] all columns:  Index(['id_str', 'full_text'], dtype='object')
[INFO] Preprocessing 4177 raw texts ...
[INFO] Total preprocessed sentences: 4158
[INFO] Total Tokenized lists: 4158
[INFO] Preprocessing raw texts. Done!
[INFO] Count For empty tokens:  0


In [ ]:
# Define the topic model object
tm = Topic_Model(k=ntopic, method=method)

# Fit the topic model by chosen method
tm.fit(sentences, token_lists)

[INFO] Topic Model object created for method: LDA_BERT
[INFO] Clustering embeddings ...
[INFO] Getting vector representations for LDA ...
[INFO] Getting vector representations for LDA. Done!
[INFO] LDA Vector shape: (4158, 5)
[INFO] Getting vector representations for BERT ...


Batches:   0%|          | 0/130 [00:00<?, ?it/s]

[INFO] Getting vector representations for BERT. Done!
[INFO] BERT vector shape: (4158, 768)
[INFO] Concatenating lda vector and bert vector...
[INFO] LDA_BERT vector shape : (4158, 773)
[INFO] Fitting Autoencoder on LDA_BERT vec...
[INFO] Fitting Autoencoder Done!
130/130 [==============================] - 0s 2ms/step
[INFO] Autoencoder vector shape: (4158, 32)
[INFO] Fitting Kmeans clustering model..
[INFO] Fitting Kmeans clustering model Done!
[INFO] Clustering embeddings. Done!


In [ ]:
# Evaluate using metrics
if tm.method == "LDA":
    topics = []
    for topic in tm.ldamodel.print_topics(num_topics=ntopic):
        topic_list = [re.search(r"[a-zA-z]+", w).group(0) for w in topic[1].split('+')]
        topics.append(topic_list)
else:
    topics = get_topic_words(token_lists, tm.cluster_model.labels_)

[INFO] Getting top words for each topic from clustering result


In [ ]:
topic_keywords = dict()
for idx , key in enumerate(topics):
    topic_keywords[idx] = " ".join(key) 
    print("{} - {}".format(idx, " ".join(key)))

0 - tip propaganda ukrain russian russia peopl nat year flag news
1 - tip putin russia peopl countri day time propaganda world leader
2 - tip putin russia sanction weapon invas support countri trump ukrain
3 - tip russia putin ukrain ukrainian russian news countri propaganda peopl
4 - ukrain russia peopl tip world kerenski putin money countri news


In [ ]:
labels = tm.predict(sentences, token_lists)

c = get_coherence(tm, token_lists, 'c_v')
s = get_silhouette(tm)
print('[INFO] Number of Topics:', ntopic)

[INFO] Predicting on same train data
[INFO] Calculating Coherence Score for others (lda_bert, bert, tfidf)
[INFO] Getting top words for each topic from clustering result
[INFO] Coherence Score: 0.29998767710066804
[INFO] Calculating silhouette scores for clusters..
[INFO] Silhouette scores: 0.30351853370666504
[INFO] Number of Topics: 5


In [ ]:
# visualize and save img
visualize(tm)
for i in range(tm.k):
    get_wordcloud(tm, token_lists, i)

[INFO] Calculating UMAP projection ...


ValueError: ignored

In [ ]:
predictions = documents.copy()
predictions['topic_no'] = np.NaN

# Mapping the queries with their labels
predictions['topic_no'] = labels
predictions.head()

In [ ]:
topic_keywords

In [ ]:
predictions["keywords"] = predictions["topic_no"].map(topic_keywords)
predictions.head(10)

In [ ]:
## topic volume 

predictions["topic_no"].value_counts().plot(kind="pie")

In [ ]:
if save:
    predictions.to_csv("output/full_output.csv", index=False)